In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from mne.stats import fdr_correction
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
import random 
import subprocess
import matplotlib as mpl
from matplotlib import animation

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.2)
sns.set_style('ticks')

%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.dpi'] = 200
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# samples metadata
filtered_reads_samples_filterdis  = pd.read_pickle('.../metadata.pkl')

In [ ]:
# use human/bacterial protein abundance profile to run RF
all_uniref_human_profile = glob('hp_abun/*human_cluster_filter.pkl')
all_uniref_human_profile_df = [pd.read_pickle(x) for x in all_uniref_human_profile]

# all study names
study_name = pd.DataFrame(all_uniref_human_profile)[0].str.replace('hp_abun/','').str.replace('_human_cluster_filter.pkl','').tolist()

In [ ]:
# read tuned hyperparams from gridsearch
best_para = pd.read_pickle('RF_final/best_hyperparam.pkl')
best_para = best_para.replace(np.nan, 'None')

In [ ]:
# attach "labels" to the metagenomic samples 
def return_all_factorized():
    all_df_study = []
    for i in range(len(study_name)):
        temp = all_uniref_human_profile_df[i].reset_index()\
        .merge(filtered_reads_samples_filterdis[filtered_reads_samples_filterdis.study_name\
        == study_name[i]][['sample_id','study_condition']].dropna(), right_on = 'sample_id', \
        left_on = 'index').drop('index', axis = 1).rename(columns = {'study_condition':'y'})
        temp.index = temp.sample_id
        temp.drop('sample_id', axis = 1, inplace = True)
        temp.loc[temp['y']!='control', 'y'] = 1
        temp.loc[temp['y']=='control', 'y'] = 0
        temp['y'] = temp['y'].astype('int')
        temp = temp.reset_index(drop = True)
        all_df_study.append(temp)
    return all_df_study

all_uniref_human_profile_df_fac = return_all_factorized()

In [ ]:
# caculate the enrichment scores 
def significance(contingency_table):
    odds, fisher_pvalue = stats.fisher_exact(contingency_table)
    if (contingency_table.min()>=5) and (contingency_table.mean()>=5):
            chi, chi_pvalue, _, _ = stats.chi2_contingency(contingency_table)
            return odds, chi, chi_pvalue
    else:
        return odds, np.nan, fisher_pvalue
    
def calculate_fi(data, name='study', y_label='condition', output_dir='.'):

    sample_names = data.index.tolist()
    feature_names = [c for c in data.columns if c!='y']
    X = data[feature_names].values
    y = data['y'].astype(int).values
    
    # Fisher exact test/chi2 test
    ypos_xpos = ((X>0).astype(int)[y==1]==1).sum(axis=0)
    yneg_xneg = ((X>0).astype(int)[y==0]==0).sum(axis=0)
    ypos_xneg = ((X>0).astype(int)[y==1]==0).sum(axis=0)
    yneg_xpos = ((X>0).astype(int)[y==0]==1).sum(axis=0)
    contingency_tables = np.array([[[a,b],
                                    [c,d]] 
                                   for a,b,c,d in 
                                   zip(ypos_xpos, ypos_xneg, 
                                       yneg_xpos, yneg_xneg)])

    odds, chi ,pvalues = zip(*[significance(table) 
                               for table in contingency_tables])

    statistics = pd.DataFrame({'protein':feature_names,
                               'pos{yx}':ypos_xpos,'pos{y}':ypos_xneg, 
                               'pos{x}':yneg_xpos,'pos{}':yneg_xneg, 
                               'odds':odds,'chi':chi ,
                               'p':pvalues}).sort_values(by='p')

    statistics['LOR'] = statistics['odds'].apply(np.log)
    _, statistics['BH_FDR_threshold'] = fdr_correction(statistics['p'])
    statistics['Bonferroni_adj_p'] = statistics['p'] * statistics.shape[0]
    statistics['Bonferroni_adj_p'] = statistics['Bonferroni_adj_p'].where(statistics['Bonferroni_adj_p']<=1.0, 1.0)  
    
    real_trials   = []
    random_trials = []
    
    min_samples_leaf = best_para[best_para.study_name == name].min_samples_leaf.values[0]
    min_samples_split = best_para[best_para.study_name == name].min_samples_split.values[0]
    n_estimators = best_para[best_para.study_name == name].n_estimators.values[0]
    max_depth = best_para[best_para.study_name == name].max_depth.values[0]
    
    for i in range(100):
        if max_depth == 'None':
            clf = RandomForestClassifier(n_jobs = 10, bootstrap = True, max_features = 'auto',
                                     min_samples_leaf = min_samples_leaf,
                                    min_samples_split = min_samples_split, 
                                    n_estimators = n_estimators, class_weight = 'balanced')
        else:
            clf = RandomForestClassifier(n_jobs = 10, bootstrap = True, max_features = 'auto',
                                     min_samples_leaf = min_samples_leaf,
                                    min_samples_split = min_samples_split, 
                                    n_estimators = n_estimators, max_depth = max_depth, class_weight = 'balanced')
        
        clf.fit(X, y)
        real_trials.append(clf.feature_importances_)
        
        # Simulated Iteration
        simulated_y = [int(random.random()<=y.mean()) for i in range(len(y))]
        
        if max_depth == 'None':
            clf = RandomForestClassifier(n_jobs = 10, bootstrap = True, max_features = 'auto',
                                     min_samples_leaf = min_samples_leaf,
                                    min_samples_split = min_samples_split, 
                                    n_estimators = n_estimators, class_weight = 'balanced')
        else:
            clf = RandomForestClassifier(n_jobs = 10, bootstrap = True, max_features = 'auto',
                                     min_samples_leaf = min_samples_leaf,
                                    min_samples_split = min_samples_split, 
                                    n_estimators = n_estimators, max_depth = max_depth, class_weight = 'balanced')
            
        
        clf.fit(X, simulated_y)            
        random_trials.append(clf.feature_importances_)
        

    real = pd.DataFrame(real_trials, columns=feature_names)
    sim  = pd.DataFrame(random_trials, columns=feature_names)
    enrichment = pd.DataFrame({"real_mean":real.T.mean(axis=1), 
                               "real_std":real.T.std(axis=1),
                               "sim_mean":sim.T.mean(axis=1), 
                               "sim_std":sim.T.std(axis=1)})
    
    out_filename = '{}/{}_{}_final_{}.csv'.format(
    output_dir, name, y_label, real.shape[0])
        
    statistics.set_index('protein')[
        ['pos{yx}','pos{y}','pos{x}', 'pos{}','LOR','p','BH_FDR_threshold','Bonferroni_adj_p']
    ].join(enrichment[['real_mean','real_std','sim_mean','sim_std']]).sort_values(by='p').to_csv(out_filename)
    
    print(name, ': done')

In [ ]:
# the importance ranking and stats 
for i in range(len(study_name)):
    calculate_fi(all_uniref_human_profile_df_fac[i], name=study_name[i], 
                            output_dir='RF_final')